In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1. Focus - Pet stands out against uncluttered background, not too close / far (Питомец выделяется на незагроможденном фоне, не слишком близко/далеко).
2. Eyes - Both eyes are facing front or near-front, with at least 1 eye / pupil decently clear (Оба глаза обращены вперед или почти вперед, по крайней мере, 1 глаз/зрачок достаточно ясный).
3. Face - Decently clear face, facing front or near-front (Прилично чистое лицо, обращенное вперед или почти вперед).
4. Near - Single pet taking up significant portion of photo (roughly over 50% of photo width or height)(Один питомец занимает значительную часть фотографии (примерно более 50% ширины или высоты фотографии)).
5. Action - Pet in the middle of an action (e.g., jumping)(Домашнее животное посреди действия (например, прыжка)).
6. Accessory - Accompanying physical or digital accessory / prop (i.e. toy, digital sticker), excluding collar and leash (Сопровождающий физический или цифровой аксессуар/реквизит (например, игрушка, цифровая наклейка), за исключением ошейника и поводка).
7. Group - More than 1 pet in the photo (Больше одного питомца на фото).
8. Collage - Digitally-retouched photo (i.e. with digital photo frame, combination of multiple photos)(Фотография с цифровой ретушью (например, с цифровой фоторамкой, комбинация нескольких фотографий)).
9. Human - Human in the photo (Человек на фото).
10. Occlusion - Specific undesirable objects blocking part of the pet (i.e. human, cage or fence). Note that not all blocking objects are considered occlusion (Конкретные нежелательные объекты, блокирующие часть животного (например, человека, клетку или забор). Обратите внимание, что не все блокирующие объекты считаются перекрытием/закрытыми).
11. Info - Custom-added text or labels (i.e. pet name, description)(Пользовательский добавленный текст или метки (например, имя питомца, описание)).
12. Blur - Noticeably out of focus or noisy, especially for the pet’s eyes and face. For Blur entries, “Eyes” column is always set to 0 (Заметно не в фокусе или шумно, особенно для глаз и лица питомца. Для записей Blur в столбце «Глаза» всегда установлено значение 0).

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter("ignore")

import os
from PIL import Image

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
import seaborn as sns

In [ ]:
train_data = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
test_data = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

Сначала изучим тренировочный датасет. Целевая переменная - Pawpularity.


In [ ]:
train_data.head()

In [ ]:
train_data.info()

In [ ]:
train_data.describe()

In [ ]:
#проверка пропущенных значений
train_data.isnull().sum()

In [ ]:
#проверка NaN
isMissing = train_data.isna()
isMissing.sum().sum()

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
sns.histplot(train_data['Pawpularity'].values, bins=100, kde=True, ax=ax)
plt.title('Целевая переменная, Pawpularity', fontsize=18)
plt.xlabel('Pawpularity', fontsize=12)
plt.show()

На графике видны выбросы, избавимся от них

In [ ]:
train_data = train_data.loc[((train_data["Pawpularity"]>3) & (train_data["Pawpularity"]<100))]

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
sns.histplot(train_data['Pawpularity'].values, bins=50, kde=True, ax=ax)
plt.title('Целевая переменная (pawpularity)')
plt.xlabel('Pawpularity')
plt.show()

In [ ]:
corrs_find = train_data.corr(method='pearson')
corrs_find_paw = corrs_find["Pawpularity"][:-1]
corrs_find_paw[abs(corrs_find_paw).argsort()[::-1]]

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)}) #изменим размер графика для лучшего воприятия
sns.heatmap(train_data.corr(), annot=True, linewidths=.5)
plt.show()

Обучение моделей

In [ ]:
classifiers=[KNeighborsRegressor(10),
            RandomForestRegressor(n_estimators=200,max_depth=5,random_state=42),
            GradientBoostingRegressor(n_estimators=200,max_depth=5,random_state=42)]
rmse_dict={}
log_cols=['Algoritm','RMSE']
log=pd.DataFrame(columns=log_cols)
X_train, X_test, Y_train, Y_test = train_test_split(train_data.drop(['Id', 'Pawpularity'], axis=1), 
                                                    train_data['Pawpularity'], 
                                                    test_size=0.2, 
                                                    random_state=42)
for clf in classifiers:
    name=clf.__class__.__name__
    model_fit= clf.fit(X_train,Y_train)
    predictions=clf.predict(X_test)
    rmse=mean_squared_error(Y_test,predictions,squared=False)
    print(clf,rmse)
    if name in rmse_dict:
        rmse_dict[name]+=rmse
    else:
        rmse_dict[name]=rmse
for clf in rmse_dict:
    log_entry=pd.DataFrame([[clf,rmse_dict[clf]]],columns=log_cols)
    log=log.append(log_entry)
plt.xlabel('RMSE') 
import seaborn as sns
sns.set_color_codes('muted')
sns.barplot(x='RMSE',y='Algoritm',data=log.sort_values(by='RMSE'),color='b')
plt.show()

Лучшей моделью является RandomForestRegressor, у которой значение среднеквадратичной ошибки составляет 17.3

In [ ]:
sub = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")
sub['Pawpularity'] = model_fit.predict(test_data.drop(['Id'], axis=1))
sub.to_csv("submission.csv", index=False)
sub